# Introduction to Data Structures and Algorithms
An algorithm describes the computational steps to compute an exact answer to a problem instance. During this presentation we will cover a number of modules:

[Log(n) Behavior](#logn)

[Word Ladder Problem](#wordladder)

- [Challenge Problem #1](#challenge-find-word-ladder) Find Word Ladder between two words
- [Challenge Problem #2](#challenge-non-participate) Find a list of words that do not participate in any Word Ladder

[Basic Data Structures](#basicds)

- [Data Structures Problem #1](#ds-challenge-1) Working with Stacks
- [Data Structures Problem #2](#ds-challenge-2) Working with Queues
- [Data Structures Problem #3](#ds-challenge-3) Working with Deques

[Sorting Algorithms](#sorting)

[Graph Algorithms](#graph)

- [Challenge Problem #3](#challenge-longest) Find longest Word Ladder with given dictionary
- [Challenge Problem #4](#challenge-cliques) Find all Cliques of words forming disjoint Word Ladders



<a id='logn'></a>
## Log(n) Behavior

Let's start with a simple request, namely, to add a list of numbers

In [ ]:
numbers = [5, 2, 99, 13, 9]
total = sum(numbers)
"Total is %d" % total

What algorithm would you design to compute the sum of these numbers?

In [ ]:
numbers = [5, 2, 99, 13, 9]
total = 0

# Fill in your code here

"Total is %d" % total

OK. This wasn't that challenging of a problem. Let's try something else. What if you wanted to check whether a list  contains a target integer?

In [ ]:
target = 13

def contains(aList, tgt):
    return tgt in aList

contains(numbers, target)

Since you know nothing about the list, the above code will always work. But what if the numbers were in ascending order? There would be no need to search through whole list, but rather, you could stop as soon as you found the target you were looking for or you encountered a greater number. 

In [ ]:
def sortedContains(aList, tgt):
    for v in aList:
        if v > tgt:
            return False
        if v == tgt:
            return True
    return False        

sortedContains(numbers, target)

Wait. That didn't work! Oh, of course, the input to sortedContains was not a sorted list of numbers. Lets try again

In [ ]:
sortedNumbers = sorted(numbers)

sortedContains(sortedNumbers, target)

Can we invent a way to search for a target value in a sorted list that is more efficient than checking each number sequentially in order?

In [ ]:
def binaryArraySearch(aList, tgt):
    lo = 0
    hi = len(aList) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if tgt < aList[mid]:
            hi = mid - 1
        elif tgt > aList[mid]:
            lo = mid + 1
        else:
            return True
    return False

binaryArraySearch(sortedNumbers, target)

Each of these three functions returns the same result, but they exhibit difference performance. 

In [ ]:
import timeit
import random

def outputTiming():
    """
    Generate timing report using random integers from 0 to 16777216,
    which greatly reduces the chance that a duplicate exists, thus
    ensuring the worst case scenario. Use random.seed(trial) to try
    to produce identical number sets across different approaches.
    """
    print ('N\tContains\tSortContains\tBinaryArraySearch')
    for trial in [2**_ for _ in range(1,14)]:
        numbers = f'sorted([random.randint(0, 2 ** 24) for _ in range({trial})])'
     
        methods = ['contains', 'sortedContains', 'binaryArraySearch']
        counts = {}
        for meth in methods:
            counts[meth] = timeit.timeit(stmt=f'{meth}(numbers, numbers[-1])', number=10000,
                        setup=f'import random\nfrom __main__ import {meth}\nrandom.seed({trial})\nnumbers = {numbers}')

        results = '\t'.join(f'{counts[meth]:f}' for meth in methods)
        print (f'{trial}\t{results}')
        
outputTiming()
"DONE"

The above table shows the performance of these three methods as the problem instances double in size (shown under column N)

<a id='wordladder'></a>
## Word Ladder Problem
In the 1970s there was a famous book by Niklaus Wirth (the inventor of the language Pascal) called "Algorithms + Data Structures = Programs". The key idea is that choosing the right data structure is essential to solving programming problems.

For this presentation we consider the Word Ladder problem. Starting with a 4-letter word, transform it into a target 4-letter word by changing one letter at a time.

COLD -> CORD -> WORD -> WARD -> WARM

Let's start by loading up a list of words from a sample dictionary

In [ ]:
inFile = open("words.english.txt", "r")
wordList = inFile.read().splitlines()

"%d words in word list" % len(wordList)

We have already seen how to locate a target word in a sorted list using BinaryArraySearch. It is possible to do better using a dictionary. For our purposes this is ideal since the list of words do not change during our program.

In [ ]:
words = {}
for word in wordList:
    words[word] = 0
    
"%d words in word dictionary" % len(words)

To keep track of the processing state while computing the Word Ladder, we use a concept called a Stage. Each stage has a word and knows its previous stage

In [ ]:
class Stage:
    """A Stage in the word ladder, recording prior word (which defaults to None)."""
    def __init__(self, word, prior = None):
        self.word = word
        self.prior = prior

    def collectTrail(self):
        """Return list of words from source to end"""
        trail = []
        node = self
        while node:
            trail.insert(0, node.word)
            node = node.prior
        return trail
    
"Stage class defined"

We need to have a way to compute all neighboring words for a given word

In [ ]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWYXZ"

def neighbors(word):
    """Return valid neighboring 4-letter words of given word."""
    found = []
    for let in alphabet:
        for pos in range(0,4):
            newWord = word[0:pos] + let + word[pos+1:]
            if newWord in words and newWord != word:
                found.append(newWord)
                
    return found

print ("Here are neighbors of COLD")
','.join(neighbors('COLD'))

<a id='challenge-find-word-ladder'></a>
**Challenge Problem 1**: To pull everything together, the following code uses a queue to explore and compute the Word Ladder from `start` to `end`.

In [ ]:
def exploreQueue(start, end):
    """Using existing collection of words, find ladder from start to end"""
   
    from collections import deque
    active = deque()
    active.append(Stage(start))
    
    while active:
        st = active.popleft()
            
        for nxt in neighbors(st.word):
            link = Stage(nxt, st)
            if nxt == end:
                return link
            active.append(link)

    # No chain
    return []

stg = exploreQueue('COLD', 'WARM')
"%s is the final word in the word ladder" % stg.word

The result returned is the final stage in the word ladder (or 'WARM'). We need additional code to be able to recover the word ladder. Just invoke 'collectTrail' on the stage to retrieve the full word ladder.

In [ ]:
stg.collectTrail()

<a id='challenge-non-participate'></a>
**Challenge Problem 2:** What if you wanted to find a list of words that do not participate in any Word Ladder? The following shows an inefficient solution that takes about 45 seconds using `slowNeighbors`.  Change it to `neighbors` and observe how much faster the program runs

In [ ]:
notParticipating = []

def slowNeighbors(word):
    """When only wordList available."""
    found = []
    for let in alphabet:
        for pos in range(0,4):
            newWord = word[0:pos] + let + word[pos+1:]
            if newWord in wordList and newWord != word:
                found.append(newWord)
                
    return found

for word in wordList: 
    if len(slowNeighbors(word)) == 0:        ### Change 'slowNeighbors' to 'neighbors' to see how much faster it runs!
           notParticipating.append(word)

','.join(notParticipating)                   ### 60 words belong to no word ladder

<a id='basicds'></a>
## Basic Data Structures

In this presentation we cover a number of fundamental data structures provided for you in Python's default libraries. These are Stacks, Queues, and Double-ended Queues.

In [ ]:
st = []
st.append(27)
print (st.pop())

<a id='ds-challenge-1'></a>
**Data Structures Challenge 1**: What operations would you do to convert the stack 'st1' into the stack '[3, 4, 8, 9]'. Run the following block first and try to issue commands to st1.

In [ ]:
st1 = []
st1.append(3)
st1.append(4)
st1.append(6)
st1.append(7)
st1

In [ ]:
# Enter commands that transforms st1 into the stack '[3, 4, 8, 9]'


# See if it worked...
"Did you succeed? %r" % (st1 == [3, 4, 8,9])

<a id='ds-challenge-2'></a>
**Data Structures Challenge 2**: What operations would you do to convert the queue 'q1' into the queue '[8, 3, 5, 6]'. Note for this exercise you must run the code block below to recreate queue if you fail

In [ ]:
from queue import Queue
q1 = Queue()
q1.put(2)
q1.put(8)
q1.put(3)

"Queue has %d elements" % q1.qsize()

In [ ]:
# Enter commands that transforms q1 into the queue '[8, 3, 5, 6]'


# If this fails, you must run block above
if q1.qsize() == 4:
    print("Did you succeed? %r" % ([q1.get(), q1.get(), q1.get(), q1.get()] == [8, 3, 5, 6]))
else:
    print ("Did not succeed")

<a id='ds-challenge-3'></a>
**Data Structures Challenge 3**: What operations would you do to convert the deque 'dq1' into the deque '[1, 3, 4]'. Note for this exercies you must run the code block below to recreate deque if you fail

In [ ]:
from collections import deque
dq1 = deque()
dq1.append(7)
dq1.append(1)
dq1.append(3)

dq1

In [ ]:
# Enter commands that transforms dq1 into the deque '[1, 3, 4]'


# If this fails, you must run block above
if len(dq1) == 3:
    print("Did you succeed? %r" % ([dq1.popleft(), dq1.popleft(), dq1.popleft()] == [1, 3, 4]))
else:
    print ("Did not succeed")

<a id='sorting'></a>
## Sorting
Sorting is a fundamental problem in computer science and a practical application in most programs. The goal is to take a list of objects and sort them in place according to some criteria (typically ascending values).

In [ ]:
def insertionSort (A):
    """ Apply INSERTION SORT on A."""
    for i in range(1,len(A)):
        print ("Iteration %d : %r" % (i, A))
        pos = i-1
        val = A[i]
        while pos >= 0 and A[pos] > val:
            A[pos+1] = A[pos]
            pos -= 1
        A[pos+1] = val
        
insertExample = [8, 1, 11, 4, 15, 9, 7, 12, 13, 6]
insertionSort(insertExample)
insertExample

When you run the above code block you can see that the array is sorted from left to right iteratively.

Can we do better? One inspiration is to subdivide the problem into two half-sized problems.  The solution, called MERGE SORT, depends on the ability to merge two sorted sub-lists, A[lo..mid] and A[mid+1..hi] into A in place. The following merge function does thius, using an auxiliary storage list of sufficient size.

In [ ]:
def merge(A, aux, lo, mid, hi):
    """Merge A[lo:hi+1] and leave result in A using aux for storage."""
    for k in range(lo, hi+1):
        aux[k] = A[k]

    i = lo
    r = mid+1

    # merge is now ready. Merge from aux back into A
    for k in range(lo, hi+1):
        if i > mid:            
            A[k] = aux[r]
            r += 1
        elif r > hi:
            A[k] = aux[i]
            i += 1
        elif aux[r] < aux[i]:
            A[k] = aux[r]
            r += 1
        else:
            A[k] = aux[i]
            i += 1
            
m_aux = [0,0,0,0,0,0,0,0]

# Note that the first four values are in ascending order (1,5,6,7) and the last four (2, 3, 4, 8)
mExample = [1, 5, 6, 7, 2, 3, 4, 8]
merge(mExample, m_aux, 0, 3, 7)
mExample

With the above merge capability, then merge sort can be quickly defined:

In [ ]:
def mergeSort (A):
    msort (A, [None] * len(A), 0, len(A)-1)

def msort(A, aux, lo, hi):
    if hi > lo:
        mid = (lo + hi) // 2

        msort(A, aux, lo, mid)
        msort(A, aux, mid+1, hi)
        
        merge(A, aux, lo, mid, hi)
        
mSortExample = [1, 5, 6, 7, 2, 3, 4, 8]
mergeSort(mSortExample)
mSortExample

<a id='graph'></a>
## Graph Data Structure
Graphs are fundamental to computer science and many real-world problems. The code in this section relies on NetworkX, a freely available and powerful third-party library for graph algorithms.

Install [NetworkX](https://networkx.github.io/) as follows:

```
pip3 install networkx==2.3
```



In [ ]:
import networkx as nx
G = nx.DiGraph()

G.add_edge('A', 'B')
G.add_edge('B', 'A')
G.add_edge('C', 'A')
G.add_edge('B', 'C')

print ("Nodes %r" % G.nodes())
print ("Edges %r" % G.edges())

You can also emit visualizations from a graph if you have matplotlib installed (note: you may have to execute the following block twice because of deprecated usage within NetworkX)

In [ ]:
import matplotlib.pyplot as plt

nx.draw(G, with_labels=True)
plt.savefig("simple_graph.png") 

Let's construct a graph where each node represents a four-letter word. How should we define the edges on the graph? One idea is to have an edge between nodes U and V if their corresponding words are neighbors, that is, are different by exactly one letter.  So the nodes "COLD" and "BOLD" will have an edge between them, since they have three letters in common.

In [ ]:
import networkx as nx

def createGraph(allWords):
    """Pass in a list of four-letter words."""
    G = nx.Graph()
    for w in allWords:
        G.add_node(w)

    # Now create edges by considering neighbors of each word. Use fast neighbors implementation!
    for node in list(G.nodes()):
        for n in neighbors(node):
            if n > node:
                G.add_edge(n, node)

    return G

wordGraph = createGraph(wordList)
"Word Graph has %d nodes and %d edges" % (len(wordGraph.nodes()), len(wordGraph.edges()))

What would this graph of 5,875 nodes look like? Instead of computing it here on the fly, I am showing the result of executing the following statements:

```python
import networkx as nx
import matplotlib.pyplot as plt
nx.draw(wordGraph, with_labels=True)
plt.savefig("word_ladder.png")
```

![images/word_ladder.png](images/word_ladder.png)

The innermost "blob" seems to contain the vast majority of the nodes, while orbitting is an "asteroid belt" of nodes that appear to belong to no Word Ladder. Recall how earlier we computed 60 words that belong to no word ladder? There are nodes in this asteroid belt that represent these words. Let's remove these 60 words. They are represented by nodes that have no edges, or in graph terms, they have a degree of 0.

In [ ]:
nodesToRemove = []
for node in list(wordGraph.nodes()):
    if wordGraph.degree[node] == 0:
        wordGraph.remove_node(node)
        
"%d nodes remaining" % len(wordGraph.nodes())

Again, we could recreate the graph using the following statements, but I am just providing the result of this computation.

```python
plt.clf()                                # clear the old figure
nx.draw(wordGraph, with_labels=True)
plt.savefig("word_ladder_reduced.png")
```

![images/word_ladder_reduced.png](images/word_ladder_reduced.png)

It looks like there are three smaller "islands" that are not connected to the main blob. These words form word ladders, but only  using words from within their cliques. How can we discover these words programmatically? I will show you how later, but in the meantime, here is the resulting visualization of the graph when these "islands" are removed:

![images/word_ladder_clique.png](images/word_ladder_clique.png)

Because of the sheer number of nodes and edges, it can still be hard to visualize this graph.

We now turn to solving Word Ladder problems over this graph. The benefit of constructing the graph once is that it can be used repeatedly for Word Ladder questions for any two pairs of words.  What is needed is a strategy to explore the graph. We start with BreadthFirstSearch.

In [ ]:
from collections import deque
class BreadthFirstSearch:
    def __init__(self, G):
        self.G = G
        self.visited = {}
        
    def trail(self, pred, end):
        """Return list of words from source to end"""
        trail = []
        node = end
        while node:
            trail.insert(0, node)
            node = pred[node]
        return trail

    def wordLadder(self, start, end):
        """Identify word ladder using BreadthFirstSearch, should it exist."""
        active = deque()
        active.append(start)

        self.visited = {}
        self.visited[start] = True
        
        pred = {}
        pred[start] = None
        while active:
            u = active.popleft()
            if u == end:
                return self.trail(pred, end)
            
            for n in self.G.neighbors(u):
                if not n in self.visited:
                    self.visited[n] = True
                    pred[n] = u
                    active.append(n)
        return None
    
bfs = BreadthFirstSearch(wordGraph)
print ("Computed Word Ladder is %r" % bfs.wordLadder('COLD', 'WARM'))
"%d nodes visited (that's %.2f percent of the graph)" % (len(bfs.visited), 100.0*len(bfs.visited)/len(bfs.G.nodes()))

What if we pursued a depth-first strategy?

In [ ]:
class DepthFirstSearch:
    def __init__(self, G):
        self.G = G
        self.visited = {}

    def trail(self, pred, end):
        """Return list of words from source to end"""
        trail = []
        node = end
        while node:
            trail.insert(0, node)
            node = pred[node]
        return trail

    def wordLadder(self, start, end):
        """Identify word ladder using DepthFirstSearch, should it exist."""
        active = deque()
        active.append(start)
        
        self.visited = {}
        self.visited[start] = True
        
        pred = {}
        pred[start] = None
        while active:
            u = active.pop()    # Treat like a stack
            
            for n in self.G.neighbors(u):
                if not n in self.visited:
                    self.visited[n] = True
                    pred[n] = u
                    if n == end:
                        return self.trail(pred, end)
            
                    active.append(n)
        return None

dfs = DepthFirstSearch(wordGraph)
print ("Computed Word Ladder is %r" % dfs.wordLadder('COLD', 'WARM'))
"%d nodes visited (that's %.2f percent of the graph)" % (len(dfs.visited), 100.0*len(dfs.visited)/len(dfs.G.nodes()))

Well, that is a surprise! DEPTH FIRST SEARCH explored a smaller percentage of the graph, yet returned a solution that is 37-times longer? I guess you get what you pay for. DEPTH FIRST SEARCH makes no guarantee on the length of the solution whereas BREADTH FIRST SEARCH will ensure that the path returned is the shortest path, in terms of the total number of edges.

<a id='challenge-longest'></a>
**Challenge Problem 3**: Find longest-possible Word Ladder between any two words.
    
Instead of checking each of the 17,254,875 word ladders (given n=5,875 words, these are the total number of unique pairs) let's solve a more challenging problem and, in the end, solve the problem we were looking for!

ALL PAIRS SHORTEST PATH computes the distance of the shortest path between any two nodes. If you run the following code block, a word ladder of longest length is returned (if there were duplicates, just one of these would have been returned). On my computer this takes about 90 seconds.

In [ ]:
def longestWordLadderCheckAPSP(G):
    """
    Use Dijkstra's shortest_path to find the longestPath that exists
    within the Graph.
    """
    results = dict(nx.all_pairs_shortest_path(G))
    longest = 0
    longestPath = []
    allNodes = list(G.nodes)
    for src in range(0, len(allNodes)-1):
        srcNode = allNodes[src]
        for target in range(src+1, len(allNodes)):
            tgtNode = allNodes[target]
            if srcNode in results and tgtNode in results[srcNode]:
                path = results[srcNode][tgtNode]
                if len(path) > longest:
                    longest = len(path)
                    longestPath = path
    
    return longestPath

longestWordLadderCheckAPSP(wordGraph)

If you do not choose to execte the above code block, the result returned is:

`'ABRI,ABSI,ASSI,ASSE,ARSE,ERSE,ELSE,ELLE,ALLE,ALME,ALMA,AMMA,AMMI,IMMI,IMMY,ISMY,ISMS'`

If you go back and review the visualized graph from before, you will find 'ABRI' at about 1 O'Clock and 'ISMS" at about 10 O'clock if considering the graph like a clockface.

<a id='challenge-cliques'></a>
**Challenge problem 4**: Programmatically find all 'cliques' in the graph, that is, a collection of nodes that represent words that can only form Word Ladders with themselves. These are also called disjoint subsets.

In [ ]:
class Explore:
    """
    Explore Graph to find Word Ladder connected subsets that are disjoint,
    such that it is impossible to form a word ladder from A to B, but you
    can form a word ladder from C to D and these are all four letter words.
    """
    def __init__(self, G):
        self.G = G
        self.sets = {}
        self.visited = {}
        self.samples = {}
        self.explore()

    def report(self):
        """Report the results of the exploration."""
        print ("Disjoint subsets of Word Ladders")
        for key in self.sets:
            print (str(key) + " -> " + str(self.sets[key]) + " with sample of " + str(self.samples[key]))
        
    def explore(self):
        """Count number of unique disjoint subsets of nodes"""
        index = 0
        for n in self.G.nodes:
            if not n in self.visited and self.G.degree(n) > 0:
                index += 1
                active = deque()
                numFound = 0
                sample = []
                active.append(n)
                
                self.visited[n] = index
                while active:
                    u = active.pop()   
                    numFound += 1
                    if numFound < 5:
                        sample.append(u)
                    for w in self.G.neighbors(u):
                        if not w in self.visited:
                            self.visited[w] = index
                            active.append(w)
                self.sets[n] = numFound
                self.samples[n] = sample
         
Explore(wordGraph).report()